# Cleaning and Preprocessing the Scopus publications related to COVID-19

For collecting the Scopus publications related to COVID-19, we used the "pybliometrics" library. It is avaliable on https://pypi.org/project/pybliometrics/.

In [ ]:
# Importing the required libraries.
import re, csv, pandas as pd, numpy as np
from pylatexenc.latex2text import LatexNodes2Text

## 1. Generating the dataframe from the raw data

In [ ]:
# Creating a dataframe from the raw data.
df_data = pd.read_csv("../../data/raw/scopus_raw.csv", header=0, dtype=object)

In [ ]:
# Checking the dataframe.
df_data.head()

In [ ]:
# Visualizing the information of dataset.
df_data.info()

## 2. Cleaning and preprocessing the dataframe

In [ ]:
# Defining the function "clean_text" to clean and preprocess any text.
def clean_text(text, has_latex=False):
    if text:
        text = re.sub(r"\u2fff(s|\s)", r"'\1", re.sub(r"\s+", " ", re.sub(r"\ufeff\.?", "", re.sub(
            r"\\\\(\’\s)?", "", str(text))))).replace("\u200b", "").replace("\ue001", "").replace(
            "\ue061", "").replace("\u202f", "").replace("\u2060", "").replace("\u200f", "").replace(
            "\u200e", "").replace("\u202c", "").replace("&#x2013;", "-").replace("&quot", "\"\"").replace(
            "\u200c", "").replace("\\u0019", "").replace("\\s", "s").replace("\u202a", "").replace(
            "\u202d", "-").replace("\u0383", "-").replace("\u20f3", "ó").replace("\u20fa", "ú").replace(
            "\u2fff", "-").strip()
        text = text.replace("TNF-alpha induced", "TNF-α induced").replace(
            "TNF-Alpha induced", "TNF-α induced").replace("TNF- ␣ induced", "TNF-α induced").replace(
            "TNF-αinduced", "TNF-α induced").replace(
            "via NF- \u242c B pathway", "via NF-κB pathway").replace(
            "via NF-kappaB pathway", "via NF-κB pathway").strip()
        if has_latex:
            text = LatexNodes2Text().latex_to_text(re.sub("\\?%", "@PER@CENT@", text)).replace("@PER@CENT@", "%")
        text = re.sub(r"\s+", " ", re.sub(r"\-{2,}", "-", re.sub(r"\s?\xad(\s|\-)?", "-", text))).replace(
            "\\", "").replace("\\%", "%").replace("()", "").replace("[]", "").strip()
        return text
    else:
        return None

In [ ]:
# Removing the invalid articles.
df_data = df_data.loc[df_data.id.notnull() & df_data.eid.notnull()]

In [ ]:
# Defining the "None" value for the "NaN" values.
df_data.replace({np.nan: None}, inplace=True)

In [ ]:
# Defining the "zero" value for the articles without numbers of citation and references.
df_data.citation_num.loc[df_data.citation_num.isnull()] = 0
df_data.ref_count.loc[df_data.ref_count.isnull()] = 0

In [ ]:
# Normalizing the feature "abstract".
df_data.abstract.loc[df_data.abstract.isnull() & df_data.description.notnull()] = df_data.description.loc[
    df_data.abstract.isnull() & df_data.description.notnull()]
df_data.abstract.loc[df_data.abstract.notnull()] = df_data.abstract.loc[df_data.abstract.notnull()].apply(
    lambda x: clean_text(x, True))

In [ ]:
# Normalizing the feature "vehicle_name".
df_data.vehicle_name.loc[df_data.conference_name.notnull() & df_data.vehicle_name.notnull()] = df_data.conference_name.loc[df_data.conference_name.notnull() & df_data.vehicle_name.notnull()]
df_data.vehicle_name.loc[df_data.vehicle_name.notnull()] = df_data.vehicle_name.loc[
    df_data.vehicle_name.notnull()].apply(clean_text)

In [ ]:
# Normalizing the feature "title".
df_data.title.loc[df_data.title.notnull()] = df_data.title.loc[df_data.title.notnull()].apply(clean_text)

In [ ]:
# Removing unnecessary columns.
columns_drop = ["eid", "pii", "description", "isbn", "conf_location", "conference_name",
    "vehicle_address", "title_edition"]
df_data.drop(axis=1, columns=columns_drop, inplace=True)

In [ ]:
# Changing the type of some features.
df_data.loc[:, ["citation_num", "ref_count"]] = df_data.loc[
    :, ["citation_num", "ref_count"]].astype(np.float32)
df_data.auth_keywords.loc[df_data.auth_keywords.notnull()] = df_data.auth_keywords.loc[
    df_data.auth_keywords.notnull()].apply(eval)
df_data.index_terms.loc[df_data.index_terms.notnull()] = df_data.index_terms.loc[
    df_data.index_terms.notnull()].apply(eval)
df_data.affiliations.loc[df_data.affiliations.notnull()] = df_data.affiliations.loc[
    df_data.affiliations.notnull()].apply(eval)
df_data.subject_areas.loc[df_data.subject_areas.notnull()] = df_data.subject_areas.loc[
    df_data.subject_areas.notnull()].apply(eval)
df_data.authors.loc[df_data.authors.notnull()] = df_data.authors.loc[df_data.authors.notnull()].apply(eval)
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(eval)
df_data.references.loc[df_data.references.notnull()] = df_data.references.loc[
    df_data.references.notnull()].apply(eval)
df_data.publication_date = pd.to_datetime(df_data.publication_date)

In [ ]:
# Creating the feature "period" from the feature "publication_date".
if "period" not in df_data:
    df_data["period"] = df_data.publication_date.apply(lambda x: "{}-{}".format(x.year, x.month))

In [ ]:
# Normalizing the itens contained in the features "auth_keywords" and "index_terms".
df_data.auth_keywords.loc[df_data.auth_keywords.notnull()] = df_data.auth_keywords.loc[
    df_data.auth_keywords.notnull()].apply(lambda x: tuple([clean_text(item) for item in x]))
df_data.index_terms.loc[df_data.index_terms.notnull()] = df_data.index_terms.loc[
    df_data.index_terms.notnull()].apply(lambda x: tuple([clean_text(item) for item in x]))

In [ ]:
# Checking there are invalid values in the features "auth_keywords", "index_terms" and "subject_areas".
for column in ["auth_keywords", "index_terms", "subject_areas"]:
    count = df_data.loc[df_data[column].notnull(), column][
                [np.any([item == None or item.lower() == "none" for item in items])
                 for items in df_data.loc[df_data[column].notnull(), column]]].size
    print("{}: {}".format(column, count))

In [ ]:
# Removing the invalid values in the features "auth_keywords", "index_terms" and "subject_areas".
for column in ["auth_keywords", "index_terms", "subject_areas"]:
    df_data.loc[df_data[column].notnull(), column] = [
        tuple([item for item in items if item])
        for items in df_data.loc[df_data[column].notnull(), column]]
    df_data.loc[df_data[column].notnull(), column] = df_data.loc[
        df_data[column].notnull(), column].apply(lambda x: x if len(x) > 0 else None)

In [ ]:
# Normalizing the content contained in the features "authors", "affiliations" and "author_affil".
df_data.affiliations.loc[df_data.affiliations.notnull()] = df_data.affiliations.loc[
    df_data.affiliations.notnull()].apply(lambda x: tuple([{"id": item["id"],
        "affiliation": clean_text(item["affiliation"]), "country": item["country"]}
        for item in x if item["id"]]))
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(lambda x: tuple(
        [{"id": item["id"], "name": clean_text(item["name"]), "affil_id": item["affil_id"],
          "affiliation": clean_text(item["affiliation"]), "country": item["country"]}
         for item in x if item["id"] or item["name"] or item["affil_id"] or \
             item["affiliation"] or item["country"]]))
df_data.authors.loc[df_data.authors.notnull()] = df_data.authors.loc[
    df_data.authors.notnull()].apply(lambda x: tuple(
        [{"id": item["id"], "name": clean_text(item["name"])} for item in x if item["id"]]))

In [ ]:
# Removing the invalid values in the features "authors", "affiliations" and "author_affil".
for column in ["authors", "affiliations", "author_affil"]:
    df_data.loc[df_data[column].notnull(), column] = df_data.loc[
        df_data[column].notnull(), column].apply(lambda x: x if len(x) > 0 else None)

In [ ]:
# Creating the affiliations' and authors' IDs for those that have not a ID.
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(lambda x: tuple([{
        "id": item["id"] if item["id"] and item["name"] else \
            str(hash("{} - {}".format(item["name"], "Scopus"))) if item["name"] else None,
        "name": item["name"],
        "affil_id": item["affil_id"] if item["affil_id"] and item["affiliation"] else \
            str(hash("{} - {}".format(item["affiliation"], "Scopus"))) \
                if item["affiliation"] else None,
        "affiliation": item["affiliation"], "country": item["country"]}
    for item in x]))

In [ ]:
# Removing duplicates within the list of affiliations and authors.
df_data.author_affil.loc[df_data.author_affil.notnull()] = [
    set([(au["id"], au["name"], au["affil_id"],
        au["affiliation"], au["country"]) for au in row])
    for row in df_data.author_affil[df_data.author_affil.notnull()]]
df_data.author_affil.loc[df_data.author_affil.notnull()] = [tuple([dict(zip(
        ["id", "name", "affil_id", "affiliation", "country"], au)) for au in row])
    for row in df_data.author_affil[df_data.author_affil.notnull()]]

In [ ]:
# Removing the duplicated records by feature "id".
df_data = df_data.sort_values(by=["id", "period"]).drop_duplicates("id", keep="first")

In [ ]:
# Removing the duplicated records by features "title" and "doi".
df_data = pd.concat([df_data[df_data.title.isnull() | df_data.doi.isnull()],
    df_data[df_data.title.notnull() & df_data.doi.notnull()].sort_values(
        by=["title", "citation_num", "publication_date"]).drop_duplicates(
            ["title", "doi"], "last")], ignore_index=True)

In [ ]:
# Normalizing the feature "references".
df_data.references.loc[df_data.references.notnull()] = df_data.references.loc[
    df_data.references.notnull()].apply(lambda x: tuple(
        [{"id": ref["id"], "title": clean_text(ref["title"], True),
          "doi": clean_text(ref["doi"]), "authors": clean_text(ref["authors"], True)}
         for ref in x]))

In [ ]:
# Checking the result.
df_data.head()

In [ ]:
# Visualizing the information of dataset.
df_data.info()

## 3. Saving the dataframe

In [ ]:
# Exporting the data to CSV file.
df_data.to_csv("../../data/prepared/scopus_covid_19.csv", index=False, quoting=csv.QUOTE_ALL)